In [ ]:
#inititalize the notebook
import os
import textwrap
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
)

chat_deployment_name = "gpt-4"

print("*** init done! ***")


simple prompt

In [ ]:
question = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

response = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": question
        }
    ]
)

lines = textwrap.wrap(response.choices[0].message.content, width=80)
for line in lines:
    print(line)

system prompt

In [ ]:
question = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

response = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": question
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent die antwoord als een grappige kat"
        }
    ]
)

lines = textwrap.wrap(response.choices[0].message.content, width=80)
for line in lines:
    print(line)

RESET THE SYSTEM PROMPT

In [ ]:
firstQuestion = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

firstResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": firstQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)

lines = textwrap.wrap(firstResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

vervolgvraag

In [ ]:
secondQuestion = "Wat is het adres van de campus?"

secondResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": secondQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)

lines = textwrap.wrap(secondResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

Add more context

In [ ]:
systemWithMemory = {
    "role": "system",
    "content": f"""
            use the previous question and answer as a context 
            ##
            previous question {firstQuestion}
            previous answer {firstResponse.choices[0].message.content}
            """
}


secondResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": secondQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        },
        systemWithMemory
    ]
)

lines = textwrap.wrap(secondResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

short term memory - langchain

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

load_dotenv()


# LLM
llm = AzureChatOpenAI(
    deployment_name=chat_deployment_name,
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_API_BASE"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_type="azure",
) 


# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Je bent een behulpzame assistent"
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=False, memory=memory)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
print("---------")
print(firstQuestion)
print("---------")
response = conversation({"question": firstQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)


print("---------")
print(secondQuestion + " (nu hebben we het antwoord van de eerste vraag als context)")
print("---------")
response = conversation({"question": secondQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)




Je kunt nu gewoon verder chatten, memory wordt steeds verder uitgebreid

Let op! Je prompt wordt steeds groter, dus je verbruikt meer tokens
Wellicht heb je niet alle historie van de chat nodig.

dit is ook een voorbeeld hoe je een specifieke instructie geeft. "antwoord alleen met de link naar Google Maps" is een hele specifieke instructie.

In [ ]:
thirdQuestion = """
                    hoe zou een link naar Google Maps eruit zien voor dit adres?
                    antwoord alleen met de link naar Google Maps
                """
print("---------")
print(thirdQuestion)
print("---------")
response = conversation({"question": thirdQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)



nog een voorbeeld van een hele specifieke instructie

In [ ]:
JsonQuestion = """
                    format your output in a json format
                    output a plain json as provided in the example with the address
                    ## sample json
                    {
                        "street": "street",
                        "city": "city",
                        "country": "country",
                        "postal_code": "postal_code"
                    }
                                        
                """
print("---------")
print(JsonQuestion)
print("---------")
response = conversation({"question": JsonQuestion})

print(response["text"])

Een hele specifieke vraag

In [ ]:
mySpecificQuestion = "wanneer is de open dag?"

mySpecificResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": mySpecificQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)


lines = textwrap.wrap(mySpecificResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)



Wellicht kun je wat meer context geven in de prompt.

We gaan nu data toevoegen, hele specifieke data, namelijk een aantal pagina's
van de HSZuyd Website.

In [ ]:
#lees de pagina's om deze later toe te voegen in de prompt
page1 = open("../docs/page-0.md", "r").read()
page2 = open("../docs/page-1.md", "r").read()
page3 = open("../docs/page-2.md", "r").read()


myDataQuestion = "wanneer is de open dag?"

myDataResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": myDataQuestion
        },
        {
            "role": "system",
            "content" : f"""
                Je bent een behulpzame assistent
                Je geeft alleen antwoord op basis van de data die je hebt gekregen
                Indien je het antwoord niet kunt vinden, 
                zeg je dat je het antwoord niet weet
                
                ## data
                {page1}

                ## data
                {page2}

                ## data
                {page3}

                """
        }
    ]
)


lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

print("----")
print("Hier wat info over de tokens die je hebt gebruikt:")
print(myDataResponse.usage)

Okay....dit lijkt werken, maar we verbruiken heel veel tokens...tijd voor een volgende stap.

***Embeddings en vectoren